# Cryptopals Challenges

## Challenge 1: Convert hex to base64

In [48]:
def c1_hex_to_bytes(hex: str)-> bytes:
    return bytes.fromhex(hex)

def c1_bytes_to_hex(b: bytes)-> str:
    return bytes.hex(b)

In [49]:
import base64

def c1_hex_to_base_64(hex_input: str) -> str:
    bytes_input = c1_hex_to_bytes(hex_input)
    b64_output = base64.b64encode(bytes_input).decode()
    return b64_output

### Unit Tests for Challenge 1

In [50]:
def c1_test_hex_to_base_64():
    input_string = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
    expected_string = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
    assert(c1_hex_to_base_64(input_string) == expected_string)
    print("pass")

In [51]:
test_hex_to_base_64()

pass


## Challenge 2: Fixed XOR

In [52]:
def c2_xor_two_bytes(a: bytes, b: bytes) -> bytes:
    rtn = []

    assert(len(a) == len(b))
    
    for i in range(len(a)):
        rtn.append(a[i] ^ b[i])

    return bytes(rtn)

In [53]:
def c2_xor_two_bytes_stringify(a: str, b: str) -> str:
    return c1_bytes_to_hex(c2_xor_two_bytes(c1_hex_to_bytes(a), c1_hex_to_bytes(b)))

### Unit Tests for Challenge 2

In [54]:
def c2_test_xor_two_bytes():
    a = "1c0111001f010100061a024b53535009181c"
    b = "686974207468652062756c6c277320657965"

    c = "746865206b696420646f6e277420706c6179"
    
    assert(xor_two_bytes_stringify(a, b) == c)
    print("pass")

In [55]:
test_xor_two_bytes()

pass


## Challenge 3 - Single Byte XOR Cipher

In [70]:
def c3_score_board(input_bytes: bytes)-> float:
    if len(input_bytes) == 0:
        return 0
        
    letters_plus_space = list(range(97, 122)) + [32]

    score = 0

    for b in input_bytes:
        if b in letters_plus_space:
            score += 1

    total = len(input_bytes)

    return float(score) / float(total)

In [85]:
class c3_decrypt_result:
    def __init__(self, key, message, score):
        self.key = key
        self.message = message
        self.score = score

    def __repr__(self):
        return "message: " + str(self.message) + " score: " + str(self.score) + " key: " + str(self.key)

def c3_decrypt(cipher_text: str) -> c3_decrypt_result:
    result = None
    current_score = -1.0
    
    cipher_text_bytes = c1_hex_to_bytes(cipher_text)
    total_bytes = len(cipher_text_bytes)

    # 2^8 = 256
    for i in range(256):
        key_to_check = i.to_bytes(1, byteorder='big')
        streamed_key = key_to_check * total_bytes

        message_to_check = c2_xor_two_bytes(cipher_text_bytes, streamed_key)
        score = c3_score_board(message_to_check)

        if result is None or score > result.score:
            result = c3_decrypt_result(key_to_check, message_to_check, score)

    assert(result is not None)

    return result

### Unit Tests for Challenge 3

In [109]:
def c3_test_score_board():
    a = b"hel lo"
    b = b"!!?"
    c = b""

    assert(c3_score_board(a) == 1.0)
    assert(c3_score_board(b) == 0.0)
    assert(c3_score_board(c) == 0.0)
    print("pass")

c3_test_score_board()

pass


In [110]:
def test_c3_decrypt():
    cipher_text = '1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'
    result = c3_decrypt(cipher_text)
    print(result)

test_c3_decrypt()

message: b"Cooking MC's like a pound of bacon" score: 0.8823529411764706 key: b'X'


## Challenge 4: Detect single-character XOR

In [86]:
def c4_decrypt_best_result(lines: [str]) -> c3_decrypt_result:
    best_result = None

    for line in lines:
        result = c3_decrypt(line)

        if best_result is None or best_result.score < result.score:
            best_result = result

    assert(best_result is not None)
    return best_result


### Unit Tests for Challenge 4

In [87]:
def test_c4_decrypt_best_result():
    lines = []
    with open("challenge_data/4.txt", "r") as f:
        for line in f:
            lines.append(line.strip())
            
    result = c4_decrypt_best_result(lines)
    print(result)

test_c4_decrypt_best_result()

message: b'Now that the party is jumping\n' score: 0.9333333333333333 key: b'5'


## Challenge 5: Implement repeating-key XOR

In [98]:
def c5_repeating_key_xor(plain_text: bytes, key: bytes) -> bytes:
    # ceiling of len(plain_text) / len(key)
    streamed_key = key * (len(plain_text) // len(key) + 1)
    streamed_key = streamed_key[:len(plain_text)]
    return c2_xor_two_bytes(plain_text, streamed_key)

### Unit Tests for Challenge 5

In [108]:
def test_c5():
    plain_text = b"Burning 'em, if you ain't quick and nimble\nI go crazy when I hear a cymbal"
    key = b"ICE"
    cipher_text = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d6"\
                  "3343c2a26226324272765272a282b2f20430a652e2c652a31243"\
                  "33a653e2b2027630c692b20283165286326302e27282f"

    assert(c1_bytes_to_hex(c5_repeating_key_xor(plain_text, key)) == cipher_text)

test_c5()